In [ ]:
import requests
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup

def get_all_valid_links(url, domain=None, visited=None):
    if visited is None:
        visited = set()

    if domain is None:
        domain = urlparse(url).netloc

    if url in visited:
        return visited

    try:
        response = requests.get(url, timeout=5)
        if response.status_code != 200:
            print(f"Skipping {url} - HTTP {response.status_code}")
            return visited

        if "nothing found" in response.text.lower():
            print(f"Skipping {url} - 'Nothing Found' in content")
            return visited

        visited.add(url)
        print(f"Valid URL: {url}")

        soup = BeautifulSoup(response.content, "html.parser")
        for tag in soup.find_all("a", href=True):
            href = tag['href']
            full_url = urljoin(url, href)
            parsed_href = urlparse(full_url)

            if parsed_href.netloc == domain:
                normalized_url = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
                if normalized_url not in visited:
                    get_all_valid_links(normalized_url, domain, visited)

    except Exception as e:
        print(f"Error visiting {url}: {e}")

    return visited


In [13]:
start_url = "https://lambdalogics.com/"
all_links = get_all_valid_links(start_url)
print("\nAll internal links found:")
for link in all_links:
    print(link)


Valid URL: https://lambdalogics.com/
Valid URL: https://lambdalogics.com/about
Valid URL: https://lambdalogics.com/expertise/
Error visiting https://lambdalogics.com/expertise/cloud-services-and-devops/: HTTPSConnectionPool(host='lambdalogics.com', port=443): Read timed out. (read timeout=5)
Error visiting https://lambdalogics.com/expertise/custom-development/: HTTPSConnectionPool(host='lambdalogics.com', port=443): Read timed out.
Error visiting https://lambdalogics.com/expertise/data-engineering-services/: HTTPSConnectionPool(host='lambdalogics.com', port=443): Read timed out.
Valid URL: https://lambdalogics.com/expertise/information-security-services/
Error visiting https://lambdalogics.com/expertise/cloud-services-and-devops/: HTTPSConnectionPool(host='lambdalogics.com', port=443): Read timed out. (read timeout=5)
Error visiting https://lambdalogics.com/expertise/custom-development/: HTTPSConnectionPool(host='lambdalogics.com', port=443): Read timed out. (read timeout=5)
Error visi

In [14]:
list(all_links)

['https://lambdalogics.com/',
 'https://lambdalogics.com/expertise/information-security-services/',
 'https://lambdalogics.com/expertise/',
 'https://lambdalogics.com/careers/',
 'https://lambdalogics.com/expertise/custom-development/',
 'https://lambdalogics.com/expertise/staff-augmentation-services/',
 'https://lambdalogics.com/contact-us/',
 'https://lambdalogics.com/expertise/mobile-app-development/',
 'https://lambdalogics.com/about/',
 'https://lambdalogics.com/expertise/data-engineering-services/',
 'https://lambdalogics.com/about',
 'https://lambdalogics.com/contact',
 'https://lambdalogics.com/expertise/cloud-services-and-devops/']

In [15]:
import ollama

In [21]:
prompt = '''You are a helpful assistant. Based only on the information in the context below, answer the user's question in one or two sentences.\n    Context:\n    [Document(id='1b221c18-7336-4b9f-aca2-28dd3775ffb0', metadata={}, page_content='Careersadmin2024-08-28T03:22:37+00:00 \\n\\nLambda Logics is a leading managed IT solutions provider in the industry, offering customized technology services to businesses of all scales to maximize efficiency, minimize surprises, and increase speed to market.\\n\\nSend your inquiry'), Document(id='3bd26c24-1deb-4d3e-938f-92a643b99554', metadata={}, page_content='Arvin Leon Thorton, Trimble Inc.\\nWe started staff augmentation partnership with Lambda Logics over three years and have really benefited from their reliable services. Their commitment to open and honest communication, accountability, and integrity has been invaluable, and we anticipate continuing this fruitful collaboration. This partnership also continues to help us with our business growth.\\nIf you are a business looking to scale up, I highly recommend using Lambda Logics services.'), Document(id='cd03c7ad-ebfd-43b6-b12e-d76beec83c55', metadata={}, page_content='Affordable Solutions For Your BusinessTransform your business with Lambda Logics. Discover how you can achieve more with less through our robust IT solutions available at affordable prices.'), Document(id='7ebbcc65-3dc8-4f90-bfab-bcce530db790', metadata={}, page_content='Mona Zia, Avant Tech Pty.\\nExperience Innovation with Lambda LogicsReady to elevate IT operations and tech infrastructure to drive sustainable growth at your company? Connect with our experts today!\\nSchedule a ConsultationLeading Innovation, Shaping Tomorrow!Lamda Logics offers a range of cutting-edge, custom-tailored managed IT services to help you overcome challenges and shape your tomorrow!\\nExplore What We Do!Meet Our Client and Partners'), Document(id='46829162-a0d3-4f4c-9750-5eb47b6c67d1', metadata={}, page_content='learn moreResult-Driven – Focused – Secure\\nWhy choose Lambda Logics?We aim to be the go-to IT company for technology and consulting solutions. Here, the team thrives on long-term partnerships and successful growth as an IT consultancy group. Choose Lambda Logics for:')]\n    Question: How I contact with lambda logics?\n    Answer:'''

In [ ]:
intents = {
    'services':[
        "cloud",
        "devops",
        "infrastructure_management",
        "custom_software",
        "mobile_development",
        "staff_augmentation",
        'others'
    ],
    'company':[
        "about_us",
        "careers",
        "partners",
        "contact_us"
    ],
    'others':[
        "testimonials",
        "blog",
        "privacy_policy",
        "terms_conditions"
    ]
    
}

In [17]:
from transformers import pipeline

# Define your intent hierarchy
intents = {
    'services': [
        "cloud",
        "devops",
        "infrastructure_management",
        "custom_software",
        "mobile_development",
        "staff_augmentation",
        "others"
    ],
    'company': [
        "about_us",
        "careers",
        "partners",
        "contact_us"
    ],
    'others': [
        "testimonials",
        "blog",
        "privacy_policy",
        "terms_conditions"
    ]
}

# Load zero-shot classifier
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Function to classify both top and sub intents
def classify_intent(text, intent_tree):
    # Step 1: Classify top-level intent
    top_labels = list(intent_tree.keys())
    top_result = classifier(text, top_labels)
    top_intent = top_result['labels'][0]

    # Step 2: Classify sub-level intent
    sub_labels = intent_tree[top_intent]
    sub_result = classifier(text, sub_labels)
    sub_intent = sub_result['labels'][0]

    return {
        "top_level_intent": top_intent,
        "sub_level_intent": sub_intent,
        "confidence_scores": {
            "top_intent_score": round(top_result['scores'][0], 3),
            "sub_intent_score": round(sub_result['scores'][0], 3)
        }
    }

# Example input text
input_text = """
We help you scale your team efficiently through staff augmentation services. Our talent pool includes engineers, developers, and experts trained for enterprise-grade delivery.
"""

# Classify the input
result = classify_intent(input_text, intents)
print(result)


Device set to use cpu


{'top_level_intent': 'services', 'sub_level_intent': 'staff_augmentation', 'confidence_scores': {'top_intent_score': 0.743, 'sub_intent_score': 0.486}}


In [41]:
import ollama
import json

In [42]:
history = [
  {
    "role": "user",
    "content": "How i cennect with lambda logics?"
  },
  {
    "role": "assistant",
    "content": "+1 800 555 5555"
  },
  {
    "role": "user",
    "content": "what is this number?"
  },
  {
    "role": "assistant",
    "content": "I don't have enough information to answer this question. Can you please add or specify more information?"
  },
  {
    "role": "user",
    "content": "you send a number when i ask about how I connect with lambda logics"
  },
  
  {
    "role": "assistant",
    "content": "+1 800 555 5555"
  },

  {
    "role": "user",
    "content": "what is this number/"
  },
  {
    "role": "assistant",
    "content": "+1 800 555 5555"
  }
]

In [48]:
retrieved_context = "my name is waqar"
question = "What is my name ?   "

In [49]:
message = [
    {"role": "system", "content": "You are a helpful AI assistant. You must only answer using the information provided in the context below. If the context is empty or lacks enough information to answer the user's question, say: 'I don't have enough information to answer this question. Can you pleace add or specify more information!' and avoid including any additional explanation or message. Be clear and concise. When possible, provide a complete answer based only on the context."},
    history[-5],
    {'role': 'user', 'content': f'Context:\n"""\n{retrieved_context}'},
    {'role':'user','content': f'Question:\n"""\n{question}'},
]


In [51]:
stream = ollama.generate(
    model="llama3.2:1b",
    prompt="What is list in python",
    stream=False
    
)

In [53]:
stream['response']

'In Python, a "list" is a data structure that can be thought of as an ordered collection of items. It\'s similar to an array or a vector in other programming languages.\n\nA list in Python is defined using square brackets `[]` and elements are separated by commas. You can also add elements dynamically using the `append()` method, which adds an item to the end of the list.\n\nHere\'s an example of a simple list:\n```\nmy_list = [1, 2, 3, "hello", 4.5]\n```\nIn this example, `my_list` is a list containing five elements: the numbers 1, 2, 3, and the string "hello". The number 4.5 is also an element of the list.\n\nSome common operations you can perform on lists in Python include:\n\n* Indexing: Accessing individual elements using their index (e.g., `my_list[0]` gets the first element).\n* Slicing: Extracting a subset of elements from the list using square brackets and a range or a slice string.\n* Length: Getting the number of elements in the list.\n* Append: Adding an item to the end of 

In [ ]:
Based on the provided context, here is a concise yet thorough answer:
According to the provided context, 

In [14]:

string = "5 2 C D +"
ops = string.strip().split(' ')

In [19]:
def calPoints(ops):
    stack = []

    for op in ops:
        if op == "+":
            stack.append(stack[-1] + stack[-2])
        elif op == "D":
            stack.append(2 * stack[-1])
        elif op == "C":
            stack.pop()
        else:
            stack.append(int(op))
    
    return sum(stack)


In [42]:
line = "[13,23,12,10]"
mtx = [int(num) for num in line[1:-1].split(',')]
print(mtx)

[13, 23, 12, 10]


In [46]:
length = len(mtx)
diff_arr = []
if length == 0:
    print("Empty list")
elif length == 1:
    print("List has only one element")
else:
    for i in range(length):
        for j in range(i + 1, length):
            # print(mtx[i] , mtx[j])
            reminder = mtx[i] % 10
            print(f'reminder {mtx[i]}:', reminder)
            
                
        

reminder 13: 3
reminder 13: 3
reminder 13: 3
reminder 23: 3
reminder 23: 3
reminder 12: 2


In [20]:
calPoints(ops)

30

In [ ]:
def solve():
    import pandas as pd

    df = pd.read_csv("data/listings.csv")
    df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)

    review_cols = [
        'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
        'review_scores_checkin', 'review_scores_communication',
        'review_scores_location', 'review_scores_value'
    ]

    corr = df[review_cols + ['price']].corr()['price'].drop('price')
    result = corr.abs().idxmax()

    print(result)


In [ ]:
def solve():
    import pandas as pd

    df = pd.read_csv("data/listings.csv")
    df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)

    host_neighs = df.groupby('host_id')['neighbourhood_cleansed'].nunique()
    professional_hosts = host_neighs[host_neighs > 5].index

    df['is_professional'] = df['host_id'].isin(professional_hosts)

    mean_prices = df.groupby('is_professional')['price'].mean()
    diff = abs(mean_prices[True] - mean_prices[False])

    print(round(diff, 2))


In [ ]:
def solve():
    import pandas as pd

    df = pd.read_csv("data/listings.csv")
    df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)

    df_entire = df[df['room_type'] == 'Entire home/apt']
    df_other = df[df['room_type'] != 'Entire home/apt']

    neighborhoods = df['neighbourhood_cleansed'].unique()
    premiums = []

    for n in neighborhoods:
        med_entire = df_entire[df_entire['neighbourhood_cleansed'] == n]['price'].median()
        med_other = df_other[df_other['neighbourhood_cleansed'] == n]['price'].median()
        if not pd.isna(med_entire) and not pd.isna(med_other):
            premiums.append(med_entire - med_other)

    avg_premium = sum(premiums) / len(premiums)
    print(round(avg_premium, 2))


In [ ]:
def solve():
    import pandas as pd

    listings = pd.read_csv("data/listings.csv")
    reviews = pd.read_csv("data/reviews.csv")

    listings['price'] = listings['price'].replace('[\$,]', '', regex=True).astype(float)

    from datetime import datetime, timedelta

    reviews['date'] = pd.to_datetime(reviews['date'])
    cutoff = reviews['date'].max() - pd.DateOffset(months=12)
    reviews_recent = reviews[reviews['date'] >= cutoff]

    reviews_count = reviews_recent['listing_id'].value_counts()

    listings = listings[listings['minimum_nights'] <= 7]

    listings = listings[listings['id'].isin(reviews_count.index)]
    listings['reviews_last_year'] = listings['id'].map(reviews_count)

    listings['expected_bookings'] = listings['reviews_last_year'] / 0.6
    listings['expected_revenue'] = listings['expected_bookings'] * listings['minimum_nights'] * listings['price']

    best_listing = listings.loc[listings['expected_revenue'].idxmax()]
    print(int(best_listing['id']))


In [ ]:
def solve():
    import pandas as pd

    df = pd.read_csv("data/listings.csv")

    df = df[df['host_is_superhost'].isin(['t', 'f'])]

    means = df.groupby('host_is_superhost')['review_scores_rating'].mean()
    diff = abs(means['t'] - means['f'])

    print(round(diff, 2))


In [ ]:
def solve():
    import pandas as pd

    df = pd.read_csv("data/listings.csv")

    df['host_since'] = pd.to_datetime(df['host_since'], errors='coerce')
    df['host_since_days'] = (pd.Timestamp.now() - df['host_since']).dt.days

    df['host_identity_verified'] = df['host_identity_verified'].map({'t': 1, 'f': 0})
    df['host_is_superhost'] = df['host_is_superhost'].map({'t': 1, 'f': 0})

    attrs = [
        'host_since_days', 'host_listings_count',
        'host_identity_verified', 'calculated_host_listings_count',
        'host_is_superhost'
    ]

    corr = df[attrs + ['number_of_reviews']].corr()['number_of_reviews'].drop('number_of_reviews')
    second = corr.abs().sort_values(ascending=False).index[1]

    print(second)
